## Step 1: Configuration

Update the schema name if your Lakehouse uses a custom schema (default is `dbo`).

In [ ]:
# Configuration
# The folder where parquet files are stored in the Files section
FILES_FOLDER = "claims_data"

# Schema name (use 'dbo' for default, or your custom schema)
SCHEMA_NAME = "dbo"

# Tables to load
TABLES = [
    "claims_history",
    "claimant_profiles",
    "fraud_indicators",
    "regional_statistics",
    "policy_claims_summary",
]

print(f"Source folder: Files/{FILES_FOLDER}/")
print(f"Target schema: {SCHEMA_NAME}")
print(f"Tables to load: {len(TABLES)}")

## Step 2: Verify Source Files

Check that all parquet files exist in the Files section.

In [ ]:
import os

# List files in the source folder
base_path = f"Files/{FILES_FOLDER}"

print(f"Checking files in {base_path}...\n")

try:
    files = mssparkutils.fs.ls(base_path)
    parquet_files = [f.name for f in files if f.name.endswith('.parquet')]
    
    print(f"Found {len(parquet_files)} parquet files:")
    for f in parquet_files:
        print(f"  ✅ {f}")
    
    # Check for missing tables
    missing = [t for t in TABLES if f"{t}.parquet" not in parquet_files]
    if missing:
        print(f"\n⚠️ Missing files for tables: {missing}")
        print("Run the upload_to_fabric.py script first.")
    else:
        print("\n✅ All required files found!")
except Exception as e:
    print(f"❌ Error: {e}")
    print(f"\nMake sure the folder '{base_path}' exists and contains parquet files.")
    print("Run 'python upload_to_fabric.py' to upload the data first.")

## Step 3: Load Tables

Load each parquet file as a managed Delta table. This will:
- Read the parquet file from Files section
- Write it as a Delta table in the Tables section
- Overwrite if the table already exists

In [ ]:
from pyspark.sql import SparkSession

# Load each table
success_count = 0
failed_tables = []

for table_name in TABLES:
    file_path = f"{base_path}/{table_name}.parquet"
    full_table_name = f"{SCHEMA_NAME}.{table_name}" if SCHEMA_NAME else table_name
    
    print(f"Loading {table_name}...")
    print(f"   Source: {file_path}")
    print(f"   Target: {full_table_name}")
    
    try:
        # Read parquet file
        df = spark.read.parquet(file_path)
        row_count = df.count()
        
        # Write as Delta table (overwrites if exists)
        df.write.format("delta").mode("overwrite").saveAsTable(full_table_name)
        
        print(f" Loaded {row_count:,} rows")
        success_count += 1
        
    except Exception as e:
        print(f"   Failed: {e}")
        failed_tables.append(table_name)

print("\n" + "=" * 60)
print(f"Load complete: {success_count}/{len(TABLES)} tables")
if failed_tables:
    print(f"Failed: {', '.join(failed_tables)}")
print("=" * 60)

## Step 4: Verify Tables

Query each table to verify the data was loaded correctly.

In [ ]:
print(" Table Summary\n")
print(f"{'Table':<25} {'Rows':>10} {'Columns':>10}")
print("-" * 50)

for table_name in TABLES:
    full_table_name = f"{SCHEMA_NAME}.{table_name}" if SCHEMA_NAME else table_name
    try:
        df = spark.table(full_table_name)
        row_count = df.count()
        col_count = len(df.columns)
        print(f"{table_name:<25} {row_count:>10,} {col_count:>10}")
    except Exception as e:
        print(f"{table_name:<25} {'ERROR':>10} - {e}")

print("\n Tables are ready for the Data Agent!")

## Step 5: Sample Queries

Test a few sample queries to ensure the data is accessible.

In [ ]:
# Sample query: Claims by type
print(" Claims by Type:\n")
spark.sql(f"""
    SELECT claim_type, COUNT(*) as count, ROUND(AVG(estimated_damage), 2) as avg_damage
    FROM {SCHEMA_NAME}.claims_history
    GROUP BY claim_type
    ORDER BY count DESC
""").show()

In [ ]:
# Sample query: High-risk claimants
print(" High-Risk Claimants (risk_score > 70):\n")
spark.sql(f"""
    SELECT claimant_id, name, risk_score, claim_frequency, total_claims_count
    FROM {SCHEMA_NAME}.claimant_profiles
    WHERE risk_score > 70
    ORDER BY risk_score DESC
    LIMIT 10
""").show()

In [ ]:
# Sample query: Fraud rates by state
print(" Fraud Rates by State:\n")
spark.sql(f"""
    SELECT state, ROUND(AVG(fraud_rate), 2) as avg_fraud_rate, SUM(total_claims) as total_claims
    FROM {SCHEMA_NAME}.regional_statistics
    GROUP BY state
    ORDER BY avg_fraud_rate DESC
    LIMIT 10
""").show()

## Next Steps

1. **Open SQL Analytics Endpoint** - The tables are now accessible via SQL
2. **Create Data Agent** - Add these tables to your Fabric Data Agent
3. **Configure Agent Instructions** - Use the files in `agent_config/` folder
4. **Publish Data Agent** - Make it available for connections
5. **Create AI Foundry Connection** - Connect from Azure AI Foundry
6. **Enable in App** - Set `USE_FABRIC_DATA_AGENT=true` in `.env`